## Base de données Aout 2018
Nous allons exécuter, les mêmes commandes sur notre dataset de juillet 2018 pour avoir une base comparable

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

In [2]:
df = pd.read_csv('bdd_base.csv')
df_3 = pd.read_csv('bdd_aout.csv')
df_3 = df_3.drop(columns=['Unnamed: 0'])

In [3]:
datetime = ['order_purchase_timestamp', 'order_approved_at',
            'order_delivered_customer_date', 'order_delivered_carrier_date',
            'order_estimated_delivery_date']
for col in datetime:
    df[col] = df[col].astype('datetime64')

df[['order_purchase_timestamp', 'order_delivered_customer_date']] = df[[
    'order_purchase_timestamp', 'order_delivered_customer_date']].apply(pd.to_datetime)

In [4]:
datetime = ['order_purchase_timestamp', 'order_approved_at',
            'order_delivered_customer_date', 'order_delivered_carrier_date',
            'order_estimated_delivery_date']
for col in datetime:
    df_3[col] = df_3[col].astype('datetime64')

print(df_3.info())

df_3[['order_purchase_timestamp', 'order_delivered_customer_date']] = df_3[[
    'order_purchase_timestamp', 'order_delivered_customer_date']].apply(pd.to_datetime)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7319 entries, 0 to 7318
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       7319 non-null   object        
 1   payment_sequential             7319 non-null   int64         
 2   payment_type                   7319 non-null   object        
 3   payment_installments           7319 non-null   int64         
 4   payment_value                  7319 non-null   float64       
 5   customer_id                    7319 non-null   object        
 6   order_status                   7319 non-null   object        
 7   order_purchase_timestamp       7319 non-null   datetime64[ns]
 8   order_approved_at              7319 non-null   datetime64[ns]
 9   order_delivered_carrier_date   7319 non-null   datetime64[ns]
 10  order_delivered_customer_date  7319 non-null   datetime64[ns]
 11  order_estimated_d

In [5]:
def categorisation(x):
    if x < 3:
        return 0
    return 1


df_3['satisfaction'] = df_3['review_score'].map(lambda x: categorisation(x))

In [6]:
df_3['temps_livraisons'] = (df_3.order_delivered_customer_date -
                            df_3.order_purchase_timestamp).dt.round('1d').dt.days

df_3['retard_livraisons'] = (df_3.order_delivered_customer_date -
                             df_3.order_estimated_delivery_date).dt.round('1d').dt.days


def retard(x):
    if x < 0:
        return 0
    return x


df_3['retard_livraisons'] = df_3['retard_livraisons'].map(lambda x: retard(x))

In [7]:
def noms(x):
    if x in ['bed_bath_table', 'health_beauty', 'sports_leisure', 'furniture_decor',
             'computers_accessories', 'housewares', 'watches_gifts', 'telephony',
             'garden_tools', 'auto', 'toys', 'cool_stuff', 'perfumery', 'baby',
             'electronics']:
        return x
    return 'autres'


df_3['prodcut_category_name'] = df_3['product_category_name_english'].map(
    lambda x: noms(x))

df_3.drop("product_category_name_english", axis=1, inplace=True)

In [8]:
df_3['mois_vente'] = pd.Series(pd.Categorical(
    df['order_purchase_timestamp'].dt.month))

In [9]:
categories_client = df_3.groupby(["customer_unique_id", "prodcut_category_name"]).agg({
    'order_item_id': 'count'}).unstack()
categories_client.fillna(0, inplace=True)
categories_client.columns = categories_client.columns.droplevel(0)
categories_client["total_items"] = categories_client.sum(axis=1)

In [10]:
products_per_order = df_3.groupby(["customer_unique_id", 'order_id']).agg({
    "order_item_id": "count"})

products_per_order = products_per_order.rename(
    columns={"order_item_id": "products_per_order_mean"})

products_per_order = products_per_order.groupby("customer_unique_id")\
    .agg({"products_per_order_mean": "mean"})

In [11]:
def noms(x):
    if x in ['SP', 'RJ', 'MG', 'RS',
             'PR', 'SC']:
        return x
    return 'autres'


df_3['customer_state'] = df_3['customer_state'].map(lambda x: noms(x))

df_f = df_3.reindex(columns=["customer_unique_id",
                    "customer_state", 'mois_vente'])
df_f.head()

,customer_unique_id,customer_state,mois_vente
0,3129dce5dc566246ea1354bff5bb6fea,autres,4
1,14f64911f31f3722899596d1095a233e,MG,6
2,14f64911f31f3722899596d1095a233e,MG,12
3,14f64911f31f3722899596d1095a233e,MG,12
4,53829feb906370f277393325399b8487,SP,5


In [12]:
recurencies = df_3.groupby("customer_unique_id")\
    .agg({"order_purchase_timestamp": ["min", "max"]})
recurencies.columns = recurencies.columns.droplevel(0)

recurencies['delay_e_commande'] = (
    recurencies["max"] - recurencies["min"]).round('1d')
recurencies['max_comma'] = df_3['order_purchase_timestamp'].max()
recurencies['order_total_delais'] = (
    recurencies['max_comma'] - recurencies['max']).round('1d')

recurencies.drop(["min", "max", 'max_comma'], axis=1, inplace=True)

In [13]:
recurencies['delay_e_commande'] = recurencies['delay_e_commande'].dt.days.astype(
    'int64')
recurencies['order_total_delais'] = recurencies['order_total_delais'].dt.days.astype(
    'int64')

recurencies.drop(['delay_e_commande'], axis=1, inplace=True)

In [14]:
df_3 = df_3.rename(columns={"order_id": "nb_commandes",
                            "retard_livraisons": "retard_livraisons_mean",
                            "temps_livraisons": "temps_livraisons_mean",
                            "satisfaction": "satisfaction_mean",
                            "review_score": "review_score_mean",
                            "freight_value": "fdp_total",
                            "price": "price_total",
                            "payment_sequential": "moyen_payment_mean",
                            "payment_installments": "versement_payment_mean"
                            })

In [15]:
df_3 = df_3.groupby('customer_unique_id')\
    .agg({'nb_commandes': 'nunique',
          'retard_livraisons_mean': 'mean',
          'temps_livraisons_mean': 'mean',
          'satisfaction_mean': 'mean',
          'review_score_mean': 'mean',
          'fdp_total': 'sum',
          'price_total': 'sum',
          'moyen_payment_mean': 'mean',
          'versement_payment_mean': 'mean',
          'Year': lambda x: x.value_counts().index[0]})

df_3 = pd.merge(df_3, categories_client,
                how="left",
                on="customer_unique_id")

df_3 = pd.merge(df_3, products_per_order,
                how='left',
                on='customer_unique_id')

df_3 = pd.merge(df_3, df_f,
                how='left',
                on='customer_unique_id')

df_3 = pd.merge(df_3, recurencies,
                how='left',
                on='customer_unique_id')

In [16]:
df_3.drop_duplicates(subset="customer_unique_id", keep="first", inplace=True)

In [17]:
borne = round(np.percentile(df_3["price_total"].values, 99), 2)
df_3 = df_3.loc[df_3["price_total"] < borne]

borne = round(np.percentile(df_3["fdp_total"].values, 99), 2)
df_3 = df_3.loc[df_3["fdp_total"] < borne]

In [18]:
print("Nombre de lignes {} et {} variables".format(
    df_3.shape[0], df_3.shape[1]))

Nombre de lignes 6152 et 32 variables


# Exportation du dataset final

In [19]:
df_3.to_csv('customers-segmentation-aout.csv')